In [34]:
import pandas as pd
import numpy as np9
import os
import matplotlib.pyplot as plt
import ast
import json

In [35]:
eval_dir = 'saved_models/cifar10_uda' #usb_nlp_


In [36]:
def gather_performance_file(fname_neck):
    keep_list = []
    seed_list = []
    event_list = []
    # weight_log_list =[]
    #                  f'_meta_updating{meta_updating}_w_{w_type}_nor_{normalize_w}'
        
    for folder_name in os.listdir(eval_dir):
        # if folder_name.startswith(fname_head) and (fname_neck in folder_name):
        flag = folder_name.endswith('2') or folder_name.endswith('3') or folder_name.endswith('8') 
        
        
        if folder_name.startswith(fname_neck) and flag:
        # if fname_neck in folder_name:
            
            event_name = os.listdir(os.path.join(eval_dir,folder_name,'tensorboard'))[-1]
            path_event = os.path.join(eval_dir,folder_name,'tensorboard', event_name)
            # path_weight = os.path.join(eval_dir,folder_name,'data_weight_log.pt')
    
            keep_list.append(os.path.join(eval_dir, folder_name, 'log.txt'))
            event_list.append(path_event)
            # weight_log_list.append(path_weight)
            seed_list.append(int(folder_name.split('_')[-1]))
    # print(keep_list,len(keep_list))
    return seed_list,keep_list

In [37]:
check_metric_list = ['iteration', 'loss', 'top-1-acc']

# for avg_method in ['binary', 'macro', 'micro', 'weighted']:
#     # for metric in ['precision', 'recall', 'F1']:
#     check_metric_list.append(avg_method+"_"+metric)

In [38]:
def get_arguments_dict(arguments):
    arguments_dict = {}
    arguments_=arguments[7:]
    for key_value in arguments_[:-1].split(', '):
        key, value = key_value.split('=')
        value = value.replace("'", "")
        arguments_dict[key] = value
    return arguments_dict

In [39]:

def get_performance_table(fname_neck):
    seed_list, keep_list = gather_performance_file(fname_neck)
    performance_table_list = []
    for seed, file_path in zip(seed_list, keep_list):
        with open(file_path, 'r') as file:
            # print(seed)
            check_metric_dict, arguments_dict, performance_table = None, None, None
            drop_it = False
            for line in file:
    
                # new_training_cond = 'Arguments: Namespace'
                new_training_cond = 'use_emameta'
                check_cond = 'eval/loss'
                if new_training_cond in line:
                    if (arguments_dict is not None) and (check_metric_dict is not None) and (not drop_it):
                        performance_table = pd.DataFrame(check_metric_dict)
                        for key, value in arguments_dict.items():
                            performance_table[key] = value
                        performance_table_list.append(performance_table)
                        
                    arguments_ = line.split(new_training_cond)[1].replace('(', '{').replace(')', '}').replace("\n", "")
                    arguments_dict = get_arguments_dict(arguments_)
                check_metric_dict = {key:[] for key in check_metric_list}
    #                 print(check_metric_dict)
                if check_cond in line:
                    for metric in check_metric_dict:
                        try:
                            if metric == 'iteration':
                                metric_value = int(line.split('INFO] ')[1].split(' '+metric)[0])
                            else:
                            
                                metric_value = float(line.split(f"'eval/{metric}': ")[1].split(',')[0])
                            check_metric_dict[metric].append(metric_value)
                        except IndexError:
                            drop_it = True
                            print(f'{file_path} has missing metric')
                            break
                            
                    performance_table = pd.DataFrame(check_metric_dict)
                    for key, value in arguments_dict.items():
                        performance_table[key] = value
                    performance_table_list.append(performance_table)
    performance_complete_table = pd.concat(performance_table_list)
    return performance_complete_table

In [40]:
def draw_performance(performance_complete_table, group_list, metric_name, filter_cond=None):
    if filter_cond is not None:
        avg_metric = performance_complete_table.loc[filter_cond].groupby(group_list)[metric_name].mean()
    else:
        avg_metric = performance_complete_table.groupby(group_list)[metric_name].mean()
    plt.plot(avg_metric)
    plt.title(metric_name)
    plt.show()
    # print(avg_metric)

In [41]:
# filter_cond = performance_complete_table['optim'] == check_optim
# draw_performance(performance_complete_table, ['iteration'], 'loss', filter_cond)

In [42]:
def show_best_mean_std(performance_complete_table, group_list, metric_name, filter_cond=None):
    if filter_cond is not None:
        best_metric = performance_complete_table.loc[filter_cond].groupby(group_list)[metric_name].max()
    else:
        best_metric = performance_complete_table.groupby(group_list)[metric_name].max()
    # print(best_metric)
    best_mean = (best_metric*100).mean()
    best_std = (best_metric*100).std()
    top_5_mean = (best_metric.sort_values()[-3:]*100).mean()
    top_5_std =  (best_metric.sort_values()[-3:]*100).std()
    # print(best_metric*100)
    return f"{top_5_mean:.2f} ± {top_5_std:.2f}"
    # print(f"Best {metric_name} {best_mean:.3f} ± {best_std:.3f}",'\n', f"Top5 {metric_name} {top_5_mean:.3f} ± {top_5_std :.3f}")

In [43]:
# filter_cond = performance_complete_table['optim'] == check_optim
method_name_l=['fullysupervised_lossnet']
# b_fixmatch_lossnet_cifar10_wrn_28_2_4000_lr0.03_True_bsz100_asy_0.4_False_1000_100_mae_uni_0

dataset ='cifar10'

num_labels = 50000

# noise_l = ["sym_0.2",'sym_0.5','asy_0.4','human_aggre_0','human_random_0','human_worst_0']
noise_l = ["flip2_0",'flip2_0.2','flip2_0.4']
# noisy_ratio=0.4
# noise_l = ["sym_0",'sym_0.4','sym_0.6']
# net="wrn_28_2"
per_dict={key:{} for key in method_name_l}
# per_dict={'flexmatch_lossnet':{}}
# per_dict={'uda_lossnet':{},'fixmatch_lossnet':{},'flexmatch_lossnet':{}}
# per_dict={'fullysupervised_lossnet':{}}
# method = method_name_l[3]
meta_loss='cer'
for method in method_name_l:
# for meta_goal in meta_goal_l:
    for noise in noise_l:

        neck=f"b_{method}_{dataset}_resnet32_{num_labels}_lr0.1_True_bsz100_{noise}_False_1000_100_{meta_loss}_uni_0.001_beta0"
        
        # neck=f"b_{method}_{dataset}_wrn_28_2_{num_labels}_lr0.03_True_bsz100_{noise}_False_1000_100_mae"
    # b_fullysupervised_lossnet_cifar10_wrn_28_10_50000_lr0.05_True_bsz100_sym_0.4_False_10000_1000_feat_expno1N_uni_5e-06_beta0_13
        try:
                
            performance_complete_table = get_performance_table(neck)
            m= show_best_mean_std(performance_complete_table, ['seed'], 'top-1-acc', filter_cond=None)
            per_dict[method][noise]=m

            # per_dict[noise][meta_goal]=m
        except:
            per_dict[method][noise]=None


In [44]:
pd.DataFrame(per_dict).T

,flip2_0,flip2_0.2,flip2_0.4
fullysupervised_lossnet,92.15 ± 0.06,89.72 ± 0.50,85.43 ± 0.87
